In [120]:
#imports
import warnings
from rocketcea.cea_obj_w_units import CEA_Obj
import numpy as np
import matplotlib.pyplot as plt
from ambiance import Atmosphere
import multiprocessing
from rocketisp.geometry import Geometry
from rocketisp.stream_tubes import CoreStream
from rocketisp.efficiencies import Efficiencies
from rocketisp.nozzle.nozzle import Nozzle
from rocketisp.geometry import solidCylVol, solidFrustrumVol
from scipy.optimize import minimize
from scipy.optimize import fsolve
from scipy.interpolate import griddata
import ipywidgets as widgets
from IPython.display import display
import cantera as ct
from PIL import Image
import csv

warnings.filterwarnings("ignore")
# Check number of processors
print(f"You have {multiprocessing.cpu_count()} processors")

You have 24 processors


In [285]:
# Initialize CEA Rocket object
rocket = CEA_Obj(oxName='LOX', fuelName='RP-1', temperature_units='degK',
 cstar_units='m/sec', specific_heat_units='kJ/kg degK',
 sonic_velocity_units='m/s', enthalpy_units='J/kg',
 density_units='kg/m^3')

rocket.get_eqratio(MR=2.6)
[_, Mf] = rocket.get_SpeciesMoleFractions(MR=2.6, frozen=1)
temps = rocket.get_Temperatures(MR=2.6)
velocities = rocket.get_SonicVelocities(MR=2.6, Pc=300, frozen=1)
Mf = sorted(Mf.items(), key=lambda item: item[1], reverse=True)
for i in range(len(Mf)):
        print(f'{Mf[i][0]}: {Mf[i][1][1]*100:.6f}%')
[MW, gamma] = rocket.get_exit_MolWt_gamma(MR=2.6, frozen=1, Pc=300)
print(f'Adiabatic Flame Temperature: {temps[0]:.2f} K')
print(f'Chamber Molecular Weight: {MW:.2f} g/mol')
print(f'Chamber Specific Heat Ratio: {gamma:.2f}')
print(f'Specific Impulse: {rocket.get_Isp(MR=2.6, Pc=300,frozen=1):.2f} s')
print(f'Exit Temperature: {temps[2]:.2f} K')
print(f'Exit Mach Number: {rocket.get_MachNumber(MR=2.6, Pc=300,frozen=1):.3f}')
print(f'Exhaust Velocity: {velocities[2]:.2f} m/s')



*CO: 31.875121%
H2O: 29.435419%
*CO2: 13.432212%
*H2: 8.684783%
*OH: 7.118984%
*H: 4.636576%
*O2: 2.731485%
*O: 2.079513%
HO2: 0.004608%
HCO: 0.000563%
Adiabatic Flame Temperature: 3348.45 K
Chamber Molecular Weight: 23.11 g/mol
Chamber Specific Heat Ratio: 1.28
Specific Impulse: 324.56 s
Exit Temperature: 1890.86 K
Exit Mach Number: 4.478
Exhaust Velocity: 684.18 m/s


In [122]:
gas = ct.Solution('c12H24_cmb_mech.yaml') # Cantera object for detailed combustion analysis
fuel = 'C12H24:1'  # RP-1
oxidizer = 'O2:1'  # Oxygen
MR = 2.6 # Mixture ratio O/F

gas.set_mixture_fraction(1/(1+MR), fuel, oxidizer) # Set mixture fraction. ct uses mixture fraction as fuel / (fuel + oxidizer)
phi = gas.equivalence_ratio(fuel, oxidizer, basis='mole') # Calculate equivalence ratio
print(f'Equivalence ratio: {phi:.2f}')

Equivalence ratio: 1.32


In [112]:
# Function to define the mole fraction equations
# X: array of mole fractions (X_C02, X_CO, X_H20)
# phi: equivalence ratio
def mole_fraction_equations(X):
    equation_1 =  X[0] + X[1] + X[2] - 1  # Sum of mole fractions equals 1
    equation_2 = X[2] - X[0] - X[1]  # Ratio of Carbon to Hydrogen
    equation_3 = 4/3 * phi * X[0] + 2/3 * phi * X[1] + 2/3 * phi * X[2] - 2 * X[2]  # Ratio of Hydrogen to Oxygen
    return [equation_1, equation_2, equation_3]

X0 = [1, 1, 1]  # Initial guess for mole fractions
X_solution = fsolve(mole_fraction_equations, X0)
print(f'Mole fractions: CO2 = {X_solution[0] * 100:.3f}%, CO = {X_solution[1]*100:.3f}%, H2O = {X_solution[2]*100:.3f}%')

Mole fractions: CO2 = 13.976%, CO = 36.024%, H2O = 50.000%


In [ ]:
T_O2 = 90.170  # Assumed temperature of lox in K
T_C12H24 = 298.15 # Assumed temperature of RP-1 in K
P_c = 300 * 6894.76 # Chamber Pressure in Pa
reactants = ct.ThermoPhase('c12H24_cmb_mech.yaml')
O2 = ct.Quantity(reactants, constant='HP')
O2.TPX = T_O2, P_c, 'O2:1'
O2.moles = 18
C12H24 = ct.Quantity(reactants, constant='HP') # Set Enthalpy Constant
C12H24.TPX = T_C12H24, P_c, 'C12H24:1'
C12H24.moles = phi 
products = O2 + C12H24
products.equilibrate('HP', solver='gibbs')
sorted_mfs = sorted(products.mole_fraction_dict().items(), key=lambda x: x[1], reverse=True)
for i in range(len(sorted_mfs)):
    if sorted_mfs[i][1] > 1e-7:
        print(f'{sorted_mfs[i][0]}: {sorted_mfs[i][1]*100:.6f}%')
specific_heat_ratio = products.cp_mole/products.cv_mole
print(f'Adiabatic Flame Temperature: {products.T:.2f} K')
print(f'Chamber Molecular Weight: {products.mean_molecular_weight:.2f} g/mol')
print(f'Chamber Specific Heat Ratio: {specific_heat_ratio:.3f}')
p_exit = 10 * 6894.76 # Exit Pressure in Pa
exit_mach = ((2/(specific_heat_ratio - 1)) * ((P_c/p_exit)**((specific_heat_ratio- 1)/specific_heat_ratio) - 1))**0.5
print(f'Exit Mach Number: {exit_mach:.3f}')
exit_T = products.T * (1 + (specific_heat_ratio - 1)/2 * exit_mach**2)**-1
print(f'Exit Temperature: {exit_T:.2f} K')
exhuast_velocity = (products.cp *(products.T - exit_T))**0.5
print(f'Exhaust Velocity: {exhuast_velocity:.2f} m/s')
I_sp = exhuast_velocity / 9.81
print(f'Specific Impulse: {I_sp:.2f} s')

print(products.report())

CO: 31.886074%
H2O: 30.246662%
CO2: 13.018295%
H2: 8.886810%
OH: 7.254638%
H: 4.279640%
O2: 2.504019%
O: 1.914396%
HO2: 0.007347%
HCO: 0.001340%
H2O2: 0.000755%
CH2O: 0.000024%
Adiabatic Flame Temperature: 3541.62 K
Chamber Molecular Weight: 22.68 g/mol
Chamber Specific Heat Ratio: 1.219
Exit Mach Number: 2.774
Exit Temperature: 1920.47 K
Exhaust Velocity: 1817.52 m/s
Specific Impulse: 185.27 s

  gas:

       temperature   3541.6 K
          pressure   2.0684e+06 Pa
           density   1.5928 kg/m^3
  mean mol. weight   22.676 kg/kmol
   phase of matter   gas

                          1 kg             1 kmol     
                     ---------------   ---------------
          enthalpy       -4.1048e+05        -9.308e+06  J
   internal energy        -1.709e+06       -3.8755e+07  J
           entropy             11863        2.6902e+05  J/K
    Gibbs function       -4.2426e+07       -9.6206e+08  J
 heat capacity c_p            2037.7             46207  J/K
 heat capacity c_v         